# Lab-6 : Self-Practice

In this week self-practice, you will implement a neural network model for a regression problem. You will use the [*admission*](./Admission_Predict.csv) dataset attached, used in the previous lab



### 1. Load the dataset and do all the necessary preprocessing

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [34]:
df = pd.read_csv('Admission_Predict.csv')
df.head(5)

df = df.drop(['Serial No.'], axis=1)
df.describe()

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
count,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000
mean,316.807500,107.410000,3.087500,3.400000,3.452500,8.598925,0.547500,0.724350
std,11.473646,6.069514,1.143728,1.006869,0.898478,0.596317,0.498362,0.142609
min,290.000000,92.000000,1.000000,1.000000,1.000000,6.800000,0.000000,0.340000
25%,308.000000,103.000000,2.000000,2.500000,3.000000,8.170000,0.000000,0.640000
50%,317.000000,107.000000,3.000000,3.500000,3.500000,8.610000,1.000000,0.730000
75%,325.000000,112.000000,4.000000,4.000000,4.000000,9.062500,1.000000,0.830000
max,340.000000,120.000000,5.000000,5.000000,5.000000,9.920000,1.000000,0.970000


In [47]:
X = df.iloc[:, :-1].values
y  = df.iloc[:, -1].values

In [48]:
from sklearn.preprocessing import StandardScaler

In [49]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)

In [50]:
## Scale all the features
scaler = StandardScaler()

scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test) 

## 2. Create custom pytorch `Dataset`

You should create a class `CustomDataset` that inherits  the abstract class `torch.utils.data.Dataset` from pytorch. 

> **Note** You should overwrite `__getitem__`, supporting fetching a data sample for a given key. Subclasses could also optionally overwrite `__len__`, which is expected to return the size of the dataset by many `~torch.utils.data.Sampler` implementations and the default options of `~torch.utils.data.DataLoader`.

#### Split your dataset into train and test data loaders
You can create a `CustomDataset` instance with the entire dataframe and use [`random_split`](https://pytorch.org/docs/stable/data.html#torch.utils.data.random_split) to split it into training and testing datasets. And then, create test and train dataloader. Or you can split using `train_test_split` from sklearn and past the splitted sets to your Custom dataset class. 

Create train and test dataloader with `batch_size = 32` each

In [51]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split

class CustumData(Dataset):
    def __init__(self, X, y):
        super().__init__()
        self.y = torch.tensor(y).float()
        self.X = torch.tensor(X).float()

    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx, :], self.y[idx]

In [52]:
train_dataset = CustumData(X_train, y_train)
test_dataset = CustumData(X_test, y_test) 

train_dataloader = DataLoader(train_dataset, 32, shuffle=True)
test_dataloader = DataLoader(test_dataset, 32, shuffle=False)

In [53]:
data, label = next(iter(train_dataloader))
label.shape

torch.Size([32])

## Create the model

Using `nn`, Create a neural network with 1 hidden layers of size 100, each must be followed by a `leaky_relu` activation function and define the forward function

In [65]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self, n_hidden_unit = 100):
        super(Net, self).__init__()
        # Write 3 lines to define 3 more linear layers.
        # 2 hidden layers with number of neurons numbers: 250 and 100
        # 1 output layer that should output 10 neurons, one for each class.
        self.fc1 = nn.Linear(7, n_hidden_unit) 
        self.fc2 = nn.Linear(n_hidden_unit, 1) 

    def forward(self, x):
        # the linear layers fc1, fc2, fc3, and fc4
        # accepts only flattened input (1D batches)
        # while the batch x is of size (batch, 28 * 28)
        # define one line to flatten the x to be of size (batch_sz, 28 * 28)
        
        #x = F.sigmoid(self.fc1(x))
        #x = F.sigmoid(self.fc2(x))
        x = F.leaky_relu(self.fc1(x))
        x = F.leaky_relu(self.fc2(x)) 
        return x

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net(n_hidden_unit = 100).to(device)
#model = nn.Sequential(nn.Linear(7, 1)).to(device)

### Training loop

Define the appropriate loss function and the training loop for the training and the testing dataloader (as done in the lab). Print the final loss on the test data

In [79]:
epochs = 20
lr = 0.01
momentum = 0.5
seed = 1
log_interval = 2

loss_fn = nn.MSELoss(reduction='mean')

In [80]:
def train( model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        #print(batch_idx)
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad() 
        output = model(data).squeeze()  
        loss = loss_fn(output, target) 
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader), loss.item()))

In [81]:
def test( model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            # Do the same that was done in the previous function.
            # But without backprobagating the loss and without running the optimizers
            # As this function is only for test.
            # write 3 lines to transform the data to the device, get the output and compute the loss
            data, target = data.to(device), target.to(device)
            output = model(data).squeeze() 
            test_loss += loss_fn(output, target).item()  # sum up batch loss 

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}\n'.format(
        test_loss))

In [82]:
import torch.optim as optim

optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)


for epoch in range(1, epochs + 1):
    train(model, device, train_dataloader, optimizer, epoch)
    test(model, device, test_dataloader)


Train Epoch: 1 [0/320 (0%)]	Loss: 0.006355
Train Epoch: 1 [64/320 (20%)]	Loss: 0.004978
Train Epoch: 1 [128/320 (40%)]	Loss: 0.004816
Train Epoch: 1 [192/320 (60%)]	Loss: 0.008963
Train Epoch: 1 [256/320 (80%)]	Loss: 0.006549

Test set: Average loss: 0.0003

Train Epoch: 2 [0/320 (0%)]	Loss: 0.005448
Train Epoch: 2 [64/320 (20%)]	Loss: 0.005625
Train Epoch: 2 [128/320 (40%)]	Loss: 0.006272
Train Epoch: 2 [192/320 (60%)]	Loss: 0.004032
Train Epoch: 2 [256/320 (80%)]	Loss: 0.008954

Test set: Average loss: 0.0003

Train Epoch: 3 [0/320 (0%)]	Loss: 0.007324
Train Epoch: 3 [64/320 (20%)]	Loss: 0.007725
Train Epoch: 3 [128/320 (40%)]	Loss: 0.002788
Train Epoch: 3 [192/320 (60%)]	Loss: 0.007280
Train Epoch: 3 [256/320 (80%)]	Loss: 0.004182

Test set: Average loss: 0.0003

Train Epoch: 4 [0/320 (0%)]	Loss: 0.011262
Train Epoch: 4 [64/320 (20%)]	Loss: 0.005299
Train Epoch: 4 [128/320 (40%)]	Loss: 0.002462
Train Epoch: 4 [192/320 (60%)]	Loss: 0.005512
Train Epoch: 4 [256/320 (80%)]	Loss: 0.0045

In [83]:
y_pred_nn = model(torch.tensor(X_test).float()) 

## Compare your Neural network model to a Linear Regression
Train a simple linear regression model on the training set and print MSE on the testing set (`X_test`). Also print the MSE on the test set using the your neural model. 

> Compare the results (which performs best) and justify why

In [84]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [85]:
lr=LinearRegression()

lr.fit(X_train, y_train)

y_pred = lr.predict(X_test) 

In [86]:
print("MSE neural network", mean_squared_error(y_test, y_pred_nn.detach().numpy()))
print("MSE Linear regressionn", mean_squared_error(y_test, y_pred))

MSE neural network 0.0053836452283787155
MSE Linear regressionn 0.004580299809196732
